In [ ]:
import sqlglot
from sqlglot import parse_one, exp

In [ ]:
cte_sql = """
WITH monthly_sales AS (
  SELECT 
    DATE_TRUNC('month', o.order_date) AS month,
    c.customer_id,
    c.customer_name,
    p.product_name,
    SUM(oi.quantity * oi.unit_price) AS revenue,
    ROW_NUMBER() OVER (PARTITION BY DATE_TRUNC('month', o.order_date), c.customer_id ORDER BY SUM(oi.quantity * oi.unit_price) DESC) AS revenue_rank
  FROM 
    orders o
  JOIN 
    customers c ON o.customer_id = c.customer_id
  JOIN 
    order_items oi ON o.order_id = oi.order_id
  JOIN 
    products p ON oi.product_id = p.product_id
  WHERE 
    o.order_date >= '2024-03-01'
    AND o.order_date <= '2024-12-31'
  GROUP BY 
    DATE_TRUNC('month', o.order_date),
    c.customer_id,
    c.customer_name,
    p.product_name
),
top_sales AS (
  SELECT 
    month,
    customer_id,
    customer_name,
    product_name,
    revenue,
    revenue_rank
  FROM 
    monthly_sales
  WHERE 
    revenue_rank <= 3
),
customer_total_revenue AS (
  SELECT 
    c.customer_id,
    SUM(oi.quantity * oi.unit_price) AS total_revenue
  FROM 
    orders o
  JOIN 
    customers c ON o.customer_id = c.customer_id
  JOIN 
    order_items oi ON o.order_id = oi.order_id
  WHERE 
    o.order_date >= '2024-03-01'
    AND o.order_date <= '2024-12-31'
  GROUP BY 
    c.customer_id
),
final as (
SELECT 
  month,
  customer_id,
  customer_name,
  product_name,
  revenue
FROM 
  top_sales
JOIN 
  customer_total_revenue
ON 
  top_sales.customer_id = customer_total_revenue.customer_id
WHERE 
  customer_total_revenue.total_revenue >= 10000
ORDER BY 
  month ASC,
  revenue_rank ASC
)
select 
  *
from final;
"""

In [ ]:
sqls = ["""SELECT
EOMONTH (
     CAST(
        FORMAT(DATEADD(month, - 14, GETDATE()), 
     'yyyy-MM-01 00:00:00.000') AS datetime)) AS eom_col, 
CAST(
     FORMAT(
        DATEADD(month, - 14, GETDATE()), 
     'yyyy-MM-01 00:00:00.000') AS datetime) AS start_tm2, col3""",   
     """SELECT EOMONTH(GETDATE()) AS EndOfMonth;"""]
sql = """SELECT
    c.CustomerID,
    c.FirstName + ' ' + c.LastName AS FullName,
    c.DateOfBirth,
    FORMAT(c.AccountBalance, 'C', 'en-US') AS FormattedBalance,
    CASE
        WHEN c.Gender = 'M' THEN 'Male'
        WHEN c.Gender = 'F' THEN 'Female'
        ELSE 'Unknown'
    END AS Gender,
    o.OrderID,
    o.OrderDate,
    p.ProductName,
    p.Price,
    IIF(p.Price > 100, 'Expensive', 'Affordable') AS PriceCategory
FROM
    Customers c
    INNER JOIN Orders o ON c.CustomerID = o.CustomerID
    INNER JOIN OrderDetails od ON o.OrderID = od.OrderID
    INNER JOIN Products p ON od.ProductID = p.ProductID
WHERE
    c.DateOfBirth >= '1980-01-01'
    AND c.DateOfBirth <= '1990-12-31'
    AND o.OrderDate >= '2022-01-01'
    AND o.OrderDate <= '2022-12-31'
    AND p.CategoryID IN (1, 2, 3)
"""

In [21]:
ast = sqlglot.parse_one(sql, read='tsql')
print(ast)
# where condition
for filter in ast.find_all(exp.Where):
     print("Where Condition:", filter)

"""
for col in ast.find_all(exp.Column):
    print(col.sql())
"""
print(ast.named_selects)
Target_Column_Names = []
Source_Column_Names = []
for expression in sqlglot.parse_one(sql).find(exp.Select).args["expressions"]:
    if isinstance(expression, exp.Alias):
        # print("Alias")
        # print('Alias expreesion using txt:', expression.text("alias"))
        Target_Column_Names.append(expression.text("alias"))
        Source_Column_Names.append(repr(expression.args['this']))
        # print('alias exression:', expression.args['this'])
        # print('alias expression:', expression)
    elif isinstance(expression, exp.Column):
        # print("Column")
        #print("Only Column Name:", expression.text("this"))
        Target_Column_Names.append(expression.text("this"))
        Source_Column_Names.append(repr(expression))
        # print("Column and table name:", expression)

print("Target Columns:", Target_Column_Names)
print("Source Columns:", Source_Column_Names)

SELECT c.CustomerID, c.FirstName + ' ' + c.LastName AS FullName, c.DateOfBirth, NUMBER_TO_STR(c.AccountBalance, 'C', 'en-US') AS FormattedBalance, CASE WHEN c.Gender = 'M' THEN 'Male' WHEN c.Gender = 'F' THEN 'Female' ELSE 'Unknown' END AS Gender, o.OrderID, o.OrderDate, p.ProductName, p.Price, CASE WHEN p.Price > 100 THEN 'Expensive' ELSE 'Affordable' END AS PriceCategory FROM Customers AS c INNER JOIN Orders AS o ON c.CustomerID = o.CustomerID INNER JOIN OrderDetails AS od ON o.OrderID = od.OrderID INNER JOIN Products AS p ON od.ProductID = p.ProductID WHERE c.DateOfBirth >= '1980-01-01' AND c.DateOfBirth <= '1990-12-31' AND o.OrderDate >= '2022-01-01' AND o.OrderDate <= '2022-12-31' AND p.CategoryID IN (1, 2, 3)
Where Condition: WHERE c.DateOfBirth >= '1980-01-01' AND c.DateOfBirth <= '1990-12-31' AND o.OrderDate >= '2022-01-01' AND o.OrderDate <= '2022-12-31' AND p.CategoryID IN (1, 2, 3)
['CustomerID', 'FullName', 'DateOfBirth', 'FormattedBalance', 'Gender', 'OrderID', 'OrderDate'

In [62]:
for ast_id in ast.args["expressions"]:
    print('Columnm: ', ast_id)
    print('Column Alias:', ast_id.alias)
    print(ast.alias_or_name)

Columnm:  c.CustomerID
Column Alias: 

Columnm:  c.FirstName + ' ' + c.LastName AS FullName
Column Alias: FullName

Columnm:  c.DateOfBirth
Column Alias: 

Columnm:  NUMBER_TO_STR(c.AccountBalance, 'C', 'en-US') AS FormattedBalance
Column Alias: FormattedBalance

Columnm:  CASE WHEN c.Gender = 'M' THEN 'Male' WHEN c.Gender = 'F' THEN 'Female' ELSE 'Unknown' END AS Gender
Column Alias: Gender

Columnm:  o.OrderID
Column Alias: 

Columnm:  o.OrderDate
Column Alias: 

Columnm:  p.ProductName
Column Alias: 

Columnm:  p.Price
Column Alias: 

Columnm:  CASE WHEN p.Price > 100 THEN 'Expensive' ELSE 'Affordable' END AS PriceCategory
Column Alias: PriceCategory



In [63]:
for join_stmt in ast.find_all(exp.Join):
    print(join_stmt)
    # for from_stmt in join_stmt.parent_select.find_all(exp.From):
    #    print(from_stmt)

for from_stmt in ast.find_all(exp.From):
    print(from_stmt)
    
# Extract the columns used in the extract
for col in sqlglot.parse_one(sql).find(exp.Where).find_all(exp.Column):
    print(col)

AttributeError: 'Column' object has no attribute 'Join'

In [9]:
# not extracting the correct columns
for column in parse_one(sqls[0]).find_all(exp.Column):
    print(f"Column => {column.name}")

Column => col3
Column => month
Column => month


In [ ]:
parsed_query = parse_one(sql, dialect="tsql")
# Identify CTEs
cte_names = {cte.alias for cte in parsed_query.find_all(exp.CTE)}
# Function to check if the selection is from a CTE or an external table
def is_from_cte_or_subquery(select_expression):
    froms = select_expression.find_all(exp.Table)
    tables = [x.args.get("this").args.get("this") for x in froms]
    subqueries = select_expression.find_all(exp.Subquery)
    num_subqueries = len([_ for _ in subqueries])
    if len(tables) > 1 or num_subqueries > 0:
        return True
    # If any of the tables are in the CTE names, then it's a CTE
    if len(set(tables) & set(cte_names)) > 0:
        return True
    return False
for select in parsed_query.find_all(exp.Select):
    for selection in select.args.get("expressions", []):
        if isinstance(selection, exp.Star) and not is_from_cte_or_subquery(select):
            raise Exception(
                "select * is only allowed when selecting from a CTE.",
                select,
            )

In [10]:
from sqlglot import parse_one, exp

def filter_has_function(sql: str, dialect: str) -> list[str]:
    ast = parse_one(sql, read=dialect)
    column_function_expressions = []
    # Finds all the filters in the query
    filters = ast.find_all(exp.Where)
    for filter in filters:
        # Find all functions in the filter
        functions = filter.find_all(exp.Func)
        # Checks if the functions operates on a column
        column_function_expressions = [
            func for func in functions if list(func.find_all(exp.Column))
        ]
    return [str(func) for func in column_function_expressions]

print(filter_has_function("select * from table where month(start_date) = 5", "snowflake"))
# ['month(start_date)']

['MONTH(start_date)']


In [57]:
import sqlglot
import sqlglot.expressions as exp

query = """
select
    sales.order_id as id,
    p.product_name,
    sum(p.price) as sales_volume
from sales
right join products as p
    on sales.product_id=p.product_id
group by id, p.product_name;

"""

column_names = []

for expression in sqlglot.parse_one(query).find(exp.Select).args["expressions"]:
    if isinstance(expression, exp.Alias):
        column_names.append(expression.text("alias"))
    elif isinstance(expression, exp.Column):
        column_names.append(expression.text("this"))

print(column_names)

['id', 'product_name', 'sales_volume']


In [58]:
query_ast = parse_one(query)  
for exp in query_ast.find(exp.Group):
    print(exp)
#parse_one(query).find(exp.Order)

id
p.product_name


In [3]:
for sql in sqls:
   f_sql = sqlglot.parse_one(sql, dialect='tsql').sql(dialect='spark')
   print(f_sql)

SELECT LAST_DAY(TO_DATE(CAST(DATE_FORMAT(ADD_MONTHS(CURRENT_TIMESTAMP(), -14), 'yyyy-MM-01 00:00:00.000') AS TIMESTAMP))) AS eom_col, CAST(DATE_FORMAT(ADD_MONTHS(CURRENT_TIMESTAMP(), -14), 'yyyy-MM-01 00:00:00.000') AS TIMESTAMP) AS start_tm2, col3
SELECT LAST_DAY(TO_DATE(CURRENT_TIMESTAMP())) AS EndOfMonth


In [4]:
ast = sqlglot.parse_one(sqls[0])
ast.args

{'kind': None,
 'hint': None,
 'distinct': None,
 'expressions': [Alias(
    this=Anonymous(
      this=EOMONTH,
      expressions=[
        Cast(
          this=Anonymous(
            this=FORMAT,
            expressions=[
              Anonymous(
                this=DATEADD,
                expressions=[
                  Column(
                    this=Identifier(this=month, quoted=False)),
                  Neg(
                    this=Literal(this=14, is_string=False)),
                  Anonymous(
                    this=GETDATE)]),
              Literal(this=yyyy-MM-01 00:00:00.000, is_string=True)]),
          to=DataType(this=Type.DATETIME, nested=False))]),
    alias=Identifier(this=eom_col, quoted=False)),
  Alias(
    this=Cast(
      this=Anonymous(
        this=FORMAT,
        expressions=[
          Anonymous(
            this=DATEADD,
            expressions=[
              Column(
                this=Identifier(this=month, quoted=False)),
              Neg(
      

In [5]:
ast.args["expressions"][0]

Alias(
  this=Anonymous(
    this=EOMONTH,
    expressions=[
      Cast(
        this=Anonymous(
          this=FORMAT,
          expressions=[
            Anonymous(
              this=DATEADD,
              expressions=[
                Column(
                  this=Identifier(this=month, quoted=False)),
                Neg(
                  this=Literal(this=14, is_string=False)),
                Anonymous(
                  this=GETDATE)]),
            Literal(this=yyyy-MM-01 00:00:00.000, is_string=True)]),
        to=DataType(this=Type.DATETIME, nested=False))]),
  alias=Identifier(this=eom_col, quoted=False))

In [50]:
print(ast.args["expressions"][0])
print(ast.args["expressions"][2])
print("Alias Column Name: ", ast.args["expressions"][0].args["alias"])
print("Column: ", ast.args["expressions"][0].args["this"])
for column in ast.find_all(exp.Column):
    print(column)


EOMONTH(CAST(FORMAT(DATEADD(month, -14, GETDATE()), 'yyyy-MM-01 00:00:00.000') AS DATETIME)) AS eom_col
col3
Alias Column Name:  eom_col
Column:  EOMONTH(CAST(FORMAT(DATEADD(month, -14, GETDATE()), 'yyyy-MM-01 00:00:00.000') AS DATETIME))
col3
month
month


In [6]:
sql = """SELECT TRANSLATE(wsr.pobject_name, CHR(10)|| CHR(13), '  ') ItemRevision_object_name FROM
    infodba.pworkspaceobject wsr"""
tsql = sqlglot.transpile(sql, read="oracle", write="tsql", pretty=True)[0]
print(tsql)

SELECT
  TRANSLATE(wsr.pobject_name, CHR(10) || CHR(13), '  ') AS ItemRevision_object_name
FROM infodba.pworkspaceobject AS wsr
